# 5 Experimenting with sensor settings

In this notebook, you will experiment with a simulated equivalent to raising and lowering a downward facing light sensor on a physical robot.

The effect of changing the sensor height is modeled by changing the field of view of the sensor: the more the sensor is raised, the more it can see.

A more complex model would use a weighted average that weights contributions from pixels near the edge of the sensor less that pixels near the center of the sensor view, but our model is a simple one and all pixels in view are weighted equally.

## 5.1 Setting up the experimental environment

The *Sensor_Diameter_Test* background contains two sets of horizontal parallel lines and two sets of parallel vertical lines. The first group of lines are coloured black and have increasing thickness going down and across the page. The second group of lines also increase in thickness and are colored red.

We will use these different lines as a test bed for evaluating how different ‘height settings’ of the downward-facing light sensor affect its performance.

But first things first. Load in the simulator in the normal way:

In [ ]:
from nbev3devsim.load_nbev3devwidget import roboSim, eds

%load_ext nbev3devsim

And then preview the background in the *Simulator world* along with the *Sensor array* view, placing the robot at location `(420, 915)` and orientation `90`:

In [ ]:
%sim_magic -b Sensor_Diameter_Test -x 420 -y 915 -a 90 -HA

You should see the robot is positioned with its sensors over crossing points of some black lines; pale coloured edging creates a circular focal view in each sensor display, which makes the detected lines look like cross hairs.

## 5.1.1 Changing the height of the sensor

The sensor diameter corresponds to the width of view of the sensor, which we are using to model the height of the sensor: if the sensor has a fixed diameter, then if we lift it up it will see more of the background.

Increasing the diameter models the effect of raising the sensor, and so the display for `sensor2` appears to ‘zoom out’, and the circle looks smaller.

(Think what happens if you shine a torch against a wall. If you are far from the wall, then you may see a large blurry pool of light on the wall; as you walk towards the wall, the patch of light reduces in diameter and has sharper edges.)

A further indication that the sensor "height" has changed is that the circle on the robot that identifies the sensor has also got larger.

The *Sensor_Diameter_Config* robot configuration (which can be selected using the `--robotSetup / -r` magic switch, configures the sensors to use different sensor heights to give it a wide view. Specifically:

- the left sensor is *raised* to give it an even wider field of view;
- the right sensor is *lowered* to give it narrower field view.

Run the following code cell to configure the simulator using these settings and observe how the *Sensor array* view has now changed.

In [ ]:
%sim_magic -r Sensor_Diameter_Config -b Sensor_Diameter_Test -x 420 -y 915 -a 90 -HA

### 5.1.2 Activity – Exploring the effect of raising and lowering the sensor

Use the *Sensor_Diameter_Config* robot configuration to give the robot an eccentric eye view of the world, where one light sensor is mounted high and the other is mounted low.

The aim of this activity is to log the light sensor values as the robot drives across the test lines and see how the recorded data values compare.

Display the the light sensor values directly in the simulator chart display (click the *Chart* display toggle button, then tick *Left light* and *Right light*), or grab the data values into the notebook datalog and view them in the notebook.

If you analyse the values in the notebook, then you should make sure you clear the datalog before collecting any data from the simulator:

In [ ]:
# Clear the datalog
%sim_data --clear

Create your own program to drive the robot horizontally across the vertical bars, logging data from both light sensors as it does so. Drive the robot at least over all the black lines. Experiment with setting different speeds for the robot as it crosses the lines to see what effect, if any, that has on the readings you obtain.

Having collected the data, you can grab the datalog in to *pandas* dataframe in the notebook using some magic:

```python
df = %sim_data
```

The data in the dataframe can be visualised using the *seaborn* charting package:

```python
import seaborn as sns

# A line plot is a sensible chart type to use
# to plot the time series data
ax = sns.lineplot(x="time",
                  y="value",
                  hue='variable',
                  data=df)
```

The following code cell correctly configures the robot’s initial starting point as `(200, 645)` and orientation as `0`. Additionally, you can pass in `-r Sensor_Diameter_Config` if you want to set the sensor diameters automatically.

Download the program to the simulator then run it using the *Simulator controls* `Run` control.

In [ ]:
%%sim_magic_preloaded -b Sensor_Diameter_Test -r Sensor_Diameter_Config -AO -x 200 -y 645 -a 0
# YOUR CODE HERE


*Add your observations about the results, including the effect of the speed of the robot on the sensor values recorded. How might this effect the degree to which you can effectively control the robot?*

#### Example solution

*Click the arrow in the sidebar or run this cell to view and example program and sampled data.*

As well as viewing the data live by enabling the chart display and selecting the *Left light* and *Right light* traces, I’m also going to grab data into the datalog and then analyse it; so I’ll start by clearing the datalog.

In [ ]:
%sim_data --clear

For my program, I’m just going to drive the robot forwards, albeit relatively slowly, logging the data in an infinite while loop. As a stopping condition, I’ll break out of the while loop if I explicitly see red.

Note that in my stopping condition test, I have grabbed the sensor reading into a variable `sample` and then checked its components. If I test `colorRight.rgb[0]` and then `colorRight.rgb[1]` then there’s always a chance the `colorRight.rgb` value will have been updated between testing the first component and the second.

In [ ]:
%%sim_magic_preloaded -AOc -b Sensor_Diameter_Test -r Sensor_Diameter_Config -x 200 -y 645 -a 0 -r Sensor_Diameter_Config

#Start off with a slowish speed of 20
speed = 20

tank_drive.on(SpeedPercent(speed), SpeedPercent(speed))

while True:
    #Stopping condition on red line
    sample = colorRight.rgb
    if (sample[0]==255 and sample[1]<255):
        break
    
    intensity_left = colorLeft.reflected_light_intensity_pc
    intensity_right = colorRight.reflected_light_intensity_pc
    
    print('Light_left: ' + str(intensity_left))
    print('Light_right: ' + str(intensity_right))

    
say("All done")

I can use the following code to grab the datalog into a dataframe:

In [ ]:
#Grab the logged data into a pandas dataframe
df = %sim_data

#Preview the first few rows of the dataset
df.head()

Using `seaborn` to display a line chart of the results, I can see that the traces from each sensor seem to differ:

In [ ]:
import seaborn as sns

# A line plot is a sensible chart type to use
# to plot the time series data
ax = sns.lineplot(x="time",
                  y="value",
                  hue='variable',
                  markers=True, dashes=False,
                  data=df)

# Limit the x axis range 
# to get a better view of the chart
ax.set_xlim(0, 6);

In particular:

- the left-hand light sensor (`sensor1`, large diameter, raised high) has ‘smeared’ values and does not register very low values for the black lines
- the right-hand light sensor (`sensor2`, small diameter, closer to the background) has much ‘spikier’ values and does report much lower sensor readings when crossing the black lines.

If I speed up the robot, then the right-hand light sensor, which has a small diameter and is close to the ground, may miss the narrower black line altogether as it drives over the it.

## 5.2 Trading off physical settings and code settings

In a real physical robot, we may be able to tune elements of the physical set-up, as well as the code, to get the robot to perform as we require.

As designers and engineers, it may often be the case that a problem that is hard to solve in one domain – for example, in the creation of a computer control program – can be simplified by making changes to the physical robot.

On the other hand, we may be able to accommodate fiddly aspects of a robot’s physical set-up by simple software fixes.

We can model a similar sort of interaction in the simulator by varying elements of the robot configuration as well as program parameters.

### 5.2.1 Challenge – Exploring the effect of the light sensor diameter on the line-following behaviour (optional)

Using the `Lollipop` background and the program you previously created to complete the associated challenge, modify the light sensor setting of the sensor you have used in your program and re-run it.

Does the increased or decreased the diameter of the sensor available from the `-r Sensor_Diameter_Config` robot configuration affect the performance of the program? (Depending which side of the line you follow, you can test both sensors.)

Are there any changes you can make in the program to compensate for changes in the robot’s physical sensor configuration?

In [ ]:
%%sim_magic_preloaded --background Lollipop

# YOUR CODE HERE

*Your notes and observations here.*

### 5.2.2 Challenge – Making this robot control strategy faster (optional)

Can you tune the robot’s physical set-up to allow you to write a program that allows the robot to more quickly complete the *Lollipop* challenge?

*If you take this challenge, be careful not to spend more than 10-15 minutes on it.*

### 5.2.3 Activity – The effect of simulated noise sensors of various diameters (optional)

You have already seen how sensor noise may affect the performance of a sensor and the performance of a software control program.

If you are curious, spend ten or fifteen minutes exploring how the different levels of sensor noise affect the ranges and ‘stability’ of readings you can get for sensors of different diameters or simulated heights above the background.

## 5.3 Summary

In this notebook, you have experimented with setting the diameter of the light sensor to model raising it or lowering it and changing its field of view over the background area.

You have seen how this physical setting may influence the performance of the sensor and the ability of the robot to perceive things in its environment that are used to determine the way the robot is controlled. 

You may also have explored how this setting affects the sensitivity of the sensor relative to a particular level of noise.